**Collaborative Failtering x Knowledge-Based Model**

CF
- past user-item interactions (bookings, clicks) to infer preferences
- based on similarities between users (user-user CF) or items (item-item CF)

KBF
- uses explicit domain knowledge or content features of items(hotels) to decide on recommendations

Goal:

I need to have a subission file that ranks hotels belonging to a user search on the likeliness that the property will be booked

In [ ]:
###### DESTINATION-SPECIFIC HOTEL POPULARITY ######

""""Based on a destination, hotels that are frequantly clicked or booked in that destination are probbably relevant for future user search in that destination."""

def compute_destination_popularity(df):
    """
    Computes normalized popularity score for each hotel within a destination.
    Uses weighted booking and click signals.
    """
    pop_df = df.groupby(['srch_destination_id', 'prop_id'])[['booking_bool', 'click_bool']].sum().reset_index()

    # Weighted score: bookings more important than clicks
    pop_df['cf_score'] = 5 * pop_df['booking_bool'] + 1 * pop_df['click_bool']

    # Normalize within each destination (0 to 1)
    pop_df['cf_score'] = pop_df.groupby('srch_destination_id')['cf_score'].transform(
        lambda x: x / (x.max() + 1e-6)
    )

    return pop_df[['srch_destination_id', 'prop_id', 'cf_score']]



In [ ]:
import pandas as pd
# Step 1: Load full original training data (5M rows)
df_raw = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/training_set_VU_DM.csv")

# Step 2: Compute CF scores on full data
popularity_scores = compute_destination_popularity(df_raw)

# Step 3: Merge into your feature-engineered dataset
df_feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_candidate_feats.csv")  # or however you generate it
df_feats = df_feats.merge(popularity_scores, on=['srch_destination_id', 'prop_id'], how='left')
df_feats['cf_score'] = df_feats['cf_score'].fillna(0)


df_feats.to_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv", index=False)


In [18]:
import pandas as pd
df_feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv")
print(df_feats.columns)

Index(['srch_id', 'prop_id', 'srch_destination_id', 'price_rank',
       'star_rank_pct', 'value_score', 'price_rel_search_rank',
       'star_diff_search', 'review_diff_search', 'residual_price',
       'prop_popularity', 'dest_popularity', 'location_score_gap',
       'location_score2_gap', 'cheap_and_good', 'search_size',
       'price_per_night', 'price_per_night_rel', 'cf_score'],
      dtype='object')


In [ ]:
# od tud sam krenula ovo gore ne znam jos
test_feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_candidate_feats_test.csv")
train_feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_candidate_feats.csv")
print(test_feats.shape)
print(train_feats.shape)

(4959183, 16)
(4958347, 18)


In [10]:
train_cols = set(train_feats.columns)
test_cols = set(test_feats.columns)

print("\nIn test but NOT in train:", test_cols - train_cols)
print("In train but NOT in test:", train_cols - test_cols)


In test but NOT in train: set()
In train but NOT in test: {'dest_popularity', 'prop_popularity'}


In [21]:
df_train = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/train_imputed_CSV.csv")
# Compute popularity stats from the training set

prop_stats = df_train.groupby('prop_id')['booking_bool'].agg(['sum', 'count']).rename(columns={'sum': 'bookings', 'count': 'imps'})
dest_stats = df_train.groupby('srch_destination_id')['booking_bool'].agg(['sum', 'count']).rename(columns={'sum': 'bookings', 'count': 'imps'})

# Global booking rate for smoothing
global_booking_rate = df_train['booking_bool'].mean()

# Apply smoothing
smoothing_prop = 10
smoothing_dest = 20

prop_stats['prop_popularity'] = (prop_stats['bookings'] + smoothing_prop * global_booking_rate) / (prop_stats['imps'] + smoothing_prop)
dest_stats['dest_popularity'] = (dest_stats['bookings'] + smoothing_dest * global_booking_rate) / (dest_stats['imps'] + smoothing_dest)

# Create lookup dictionaries
prop_pop_dict = prop_stats['prop_popularity'].to_dict()
dest_pop_dict = dest_stats['dest_popularity'].to_dict()


In [20]:
df_test = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_candidate_feats_test.csv")
# Step 2: Map popularity scores
df_test['prop_popularity'] = df_test['prop_id'].map(prop_pop_dict)
df_test['dest_popularity'] = df_test['srch_destination_id'].map(dest_pop_dict)
df_test['prop_popularity'].fillna(global_booking_rate, inplace=True)
df_test['dest_popularity'].fillna(global_booking_rate, inplace=True)

C:\Users\nikak\AppData\Local\Temp\ipykernel_28756\2165746600.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['prop_popularity'].fillna(global_booking_rate, inplace=True)
C:\Users\nikak\AppData\Local\Temp\ipykernel_28756\2165746600.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [23]:
df_test.to_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_candidate_feats_test_with_pop.csv", index=False)

print("Saved updated test feature file with popularity columns.")

Saved updated test feature file with popularity columns.


**--> OKAY NOW I HAVE nika_candidate_feats.csv AND nika_candidsate_feats_test_with_pop.csv absolutely the same**

In [22]:
print(df_test.shape)
print(train_feats.shape)

train_cols = set(train_feats.columns)
test_cols = set(df_test.columns)

print("\nIn test but NOT in train:", test_cols - train_cols)
print("In train but NOT in test:", train_cols - test_cols)

(4959183, 18)
(4958347, 18)

In test but NOT in train: set()
In train but NOT in test: set()


In [27]:
cf_scores = compute_destination_popularity(df_train)

In [28]:
train_cf = train_feats.merge(cf_scores, on=['srch_destination_id', 'prop_id'], how='left')
test_cf = df_test.merge(cf_scores, on=['srch_destination_id', 'prop_id'], how='left')

In [34]:
# Fill missing cf_scores in test set (optional but recommended)
test_cf['cf_score'].fillna(0, inplace=True)

# Save to CSV
train_cf.to_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_train_feats_with_cf.csv", index=False)
test_cf.to_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_test_feats_with_cf.csv", index=False)

print("Saved:")
print("- nika_train_feats_with_cf.csv")
print("- nika_test_feats_with_cf.csv")

C:\Users\nikak\AppData\Local\Temp\ipykernel_28756\1467006053.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_cf['cf_score'].fillna(0, inplace=True)


Saved:
- nika_train_feats_with_cf.csv
- nika_test_feats_with_cf.csv


In [33]:
train_cf.columns

Index(['srch_id', 'prop_id', 'srch_destination_id', 'price_rank',
       'star_rank_pct', 'value_score', 'price_rel_search_rank',
       'star_diff_search', 'review_diff_search', 'residual_price',
       'prop_popularity', 'dest_popularity', 'location_score_gap',
       'location_score2_gap', 'cheap_and_good', 'search_size',
       'price_per_night', 'price_per_night_rel', 'cf_score'],
      dtype='object')

**This should create a final dataset for both test and train that is merged with OG data and the engineered features... lets see good luck**

In [35]:
train_imputed = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/train_imputed_CSV.csv")
feats_train = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_train_feats_with_cf.csv")

